# ETL 3: Transformación 2 - Clases y funciones de limpieza

In [7]:
#importamos las librerías:

from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast 

from datetime import datetime, timedelta

pd.options.display.max_columns=None

Abrimos los DFs:

In [8]:
df = pd.read_csv('datos/ETL-1.csv', index_col=0)
df.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 320, 'speed':...",13,15,5,1024,none,0,0,265,2,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 355, 'speed':...",12,15,5,1024,none,0,0,345,2,USA


In [9]:
df1 = pd.read_csv('datos/Limpieza-4.csv', index_col=0)
df1.head(2)

,year,type,country,activity,age,species,month,fatal,sex
0,2018,Boating,usa,Paddling,57.0,White Shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18.0,Tiger Shark,Jun,Y,M


In [10]:
class APIS: 

    def __init__(self, df, df1):

    # definimos cada una de las variables. 
        self.df = df
        self.df1 = df1

    # definimos nuestro primer método, para la llamar a la API y crear un nuevo df

    def creacion_df(self, nombre_df):

        self.nombre_df = nombre_df
        
        #almacenamos las coordinadas y los países en un diccionario:
        
        dicc_paises = {"USA": ["39.7837304", "-100.445882"], "Australia": ["-24.7761086", "134.755"], 
               "South Africa": ["-28.8166236", "24.991639"], "New Zealand": ["-41.5000831", "172.8344077"], 
               "Papua New Guinea": ["-5.6816069", "144.2489081"]}
        
        nombre_df = pd.DataFrame()
        
        #generamos un bucle for para iterar los valores del diccionario en la url:
        
        for keys, values in dicc_paises.items():
            lat = values[0]
            lon = values[1]
            country = keys
            producto = "meteo"

            response = requests.get(url=f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json')
            
            #almacenamos los resultados en un df desde un diccionario con el método 'from dict'
            df1 = pd.DataFrame.from_dict(pd.json_normalize(response.json(),["dataseries"])) #añadimos "dataseries" para que nos separe esa información por columnas
            df1["country"] = keys
            nombre_df = pd.concat([nombre_df, df1], axis = 0, ignore_index=True)
        
        return nombre_df
    
    # definimos una función para filtrar los países que nos interesan.

    def limpieza_attacks(self, df):

        self.df = df

        list_country = ['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

        df1 = df[df['country'].isin(list_country)]

        return df1

    # definimos las dos siguientes funciones para desempaquetar las columnas 'hr_profile y 'wind_profile', ya que sus valores eran diccionarios.
    #generamos un for para volcar como valores y nombres de columnas los diccionarios encerrados en el valor de las columnas

    def desempaquetar_columna_hr(self, df):

        self.df = df

        df['rh_profile'] = df['rh_profile'].apply(ast.literal_eval)
        df_clima = df['rh_profile'].apply(pd.Series)

        for i in range(len(df_clima.columns)):
         nombre_rh = "rh_"+ str(df_clima[i].apply(pd.Series)["layer"][0])
         valores_rh = (df_clima[i].apply(pd.Series)["rh"])

         df.insert(i, nombre_rh, valores_rh)

        return df
    
    #generamos un for para volcar como valores y nombres de columnas los diccionarios encerrados en el valor de las columnas

    def desempaquetar_columna_wind(self, df):

        self.df = df

        df['wind_profile'] = df['wind_profile'].apply(ast.literal_eval)
        df_wind = df['wind_profile'].apply(pd.Series)

        for i in range(len(df_wind.columns)):
         nombre_wd = "wind_direction_"+ str(df_wind[i].apply(pd.Series)["layer"][0])
         valores_wd = (df_wind[i].apply(pd.Series)["direction"])

         df.insert(i, nombre_wd, valores_wd)

        for i in range(len(df_wind.columns)):
            nombre_wd1 = "wind_speed_"+ str(df_wind[i].apply(pd.Series)["layer"][0])
            valores_wd1 = (df_wind[i].apply(pd.Series)["speed"])

            df.insert(i, nombre_wd1, valores_wd1)

        return df

    # la siguiente función borra esas mismas columnas, para eliminar la información redundante.

    def borrar_redundantes(self, df):

        self.df = df

        df.drop(['rh_profile', 'wind_profile'], axis = 1, inplace=True)

        return df

    # mergeamos las columnas desempaquetadas al df de ataques de tiburón.

    def merge_df(self, df, df1):

        self.df = df
        self.df1 = df1
    
        df['country'] = df['country'].str.lower()

        df_groupby = df.groupby(["country"])[df.columns].mean().reset_index()
        df_merge = df1.merge(df_groupby, how='outer', on = 'country', right_on= None, left_on=None)
       
        return df_merge

    # guardamos nuestro dataframe limpio.

    def guardar_csv(self, df):

        self.df = df

        df.to_csv('datos/ETL-3.csv', index=False)


### Aplicamos la clase:

Iremos concatenando los resultados de las funciones de las clases, ya que nos devolverán un dataframe modificado con el código de las funciones.

1. Almacenamos la clase en una variable.

In [11]:
clase = APIS(df, df1)

2. Creamos el df a partir de la llamada al API.

In [12]:
df_a = clase.creacion_df(df)

3. Filtramos los países que nos interesan en el df anterior.

In [13]:
df_b = clase.limpieza_attacks(df1)

4. Desempaquetamos las dos columnas que contenían un diccionario y las volcamos en nuevas columnas.

In [14]:
df_c = clase.desempaquetar_columna_hr(df)

In [15]:
df_d = clase.desempaquetar_columna_wind(df_c)

5. Borramos de nuestro df las columnas que contenían el diccionario para evitar información duplicada.

In [16]:
df_e = clase.borrar_redundantes(df_d)

6. Guardamos los datos en un csv.

In [17]:
df_f = clase.guardar_csv(df_e)